# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using

In [1]:
import numpy
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. 


[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. This has to be stored in the same folder as this file

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = "AAPL"

## Using an f-string (look at f at the start of the string) to pass in (multiple) variables in the string
## %%?token=%% is needed as authentication for IEX that we can use their API
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"

## Executing HTTP request and store the results in an outside variable
data = requests.get(api_url).json()

## data is a "Reponse" object contained within the "models" module in the "requests" library
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'KNCEO(AQLNAAADB)TGS  GEMRS S/TELL', 'calculationPrice': 'tops', 'open': None, 'openTime': None, 'openSource': 'aiofcfli', 'close': None, 'closeTime': None, 'closeSource': 'fafloiic', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 137.599, 'latestSource': 'IEX real time price', 'latestTime': '2:12:37 PM', 'latestUpdate': 1681365297639, 'latestVolume': None, 'iexRealtimePrice': 134.724, 'iexRealtimeSize': 23, 'iexLastUpdated': 1674440512338, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 137.03, 'previousVolume': 173221762, 'change': 0.005, 'changePercent': 4e-05, 'volume': None, 'iexMarketPercent': 0.01174228510860026, 'iexVolume': 663074, 'avgTotalVolume': 108592310, 'iexB

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data["latestPrice"]
market_cap = data["marketCap"]
print(market_cap)

2301304177534


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
final_df = pd.DataFrame(columns=my_columns)
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_df.append(pd.Series([symbol, price, market_cap, "N/A"],index=my_columns), ignore_index=True)

## ignore_index=True is pretty much for all append pd.Series arguments
## A Pandas Series is a one-dimensional row or column

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,137.599,2301304177534,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_df = pd.DataFrame(columns=my_columns)
for stock in stocks["Ticker"]:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    ## HTTP requests are REALLY slow, but you can do batch requests to make up for it (next section)
    data = requests.get(api_url).json()
    final_df = final_df.append(pd.Series([stock, data["latestPrice"], data["marketCap"], "N/A"], index=my_columns), ignore_index=True    
    
    

KeyboardInterrupt: 

Getting the data we need from single API calls is WAY too slow, so we stop the for loop andmake use of batch API calls to speed up the process.

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [ ]:
## Credits, Ned Batchelder: https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield succesive n-sized chunks from lst (which is list)"""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [ ]:
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_groups

symbol_strings=[]
for i in range(len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))
    ##  Appends the join method called on the COMMA character for each i in symbol_groups
    ## Join takes all elements of symbol groups and joins them together by seperating them together by COMMA
    ## Try doing: "print(symbol_strings)" to see what that means ==>
    
    #print (symbol_strings)
    
final_df = pd.DataFrame(columns=my_columns)


    ## REMEMBER: In an API call, after a question mark ?????, you have to change all question marks ???? with ampersand &&&&&&&&&&&&
for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        final_df=final_df.append(pd.Series([
                                            symbol, 
                                            ## ["quote"] is the second list argument because it is the API ENDPOINT
                                            data[symbol]["quote"]["latestPrice"],
                                            data[symbol]["quote"]["marketCap"],
                                            "N/A"
                                            ], index=my_columns), ignore_index=True)
final_df

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [ ]:
## We will now let the user input their portfolio (capital) and the script will calculate how much of each share to buy
## to own an equal weight S&P fund

## Will let the user input a value
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
    
except ValueError:
    print("That's not a number/nPlease enter an integer")
    portfolio_size = input("Enter the value of your portfolio:")
    val = float(portfolio_size)

In [ ]:
position_size=val/len(final_df.index)

for i in range(0, len(final_df.index)):
    ## Indexing in pandas. "i" becomes the row(s), "Number of Shares to Buy" becomes the column
    final_df.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_df.loc[i, "Stock Price"])
    
final_df
## math.floor is Pythons function for ROUNDING DOWN a number

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization

### Initializing our XlsxWriter Object

In [ ]:
writer=pd.ExcelWriter("recommended_trades.xlsx", engine="xlsxwriter")
final_df.to_excel(writer, "Recommended Trades", index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [ ]:
background_color = "#0a0a23"
font_color = "ffffff"

string_format = writer.book.add_format({
    "font_color": font_color,
    "bg_color": background_color,
    "border": 1 ##This means there is a SOLID border around each one
 })

dollar_format = writer.book.add_format({
    "num_format" : "$0.00", ## Sets the format as dollar and two decimals
    "font_color": font_color,
    "bg_color": background_color,
    "border": 1 
 })

integer_format = writer.book.add_format({
    "num_format" : "0",    ## Sets the format as pure integer with no decimals
    "font_color": font_color,
    "bg_color": background_color,
    "border": 1 
 })



### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [ ]:
##writer.sheets["Recommended Trades"].set_column("A:A", 18, string_format)
##writer.sheets["Recommended Trades"].set_column("B:B", 18, string_format)
##writer.sheets["Recommended Trades"].set_column("C:D", 18, string_format)
##writer.sheets["Recommended Trades"].set_column("D:D", 18, string_format)
##writer.save()


## The header columns are messed up and it is easier to just use the "string_format" we created earlier to get them right
##writer.sheets["Recommended Trades"].write("A1", "Ticker", string_format)
##writer.sheets["Recommended Trades"].write("B1", "Stock Price", dollar_format)
##writer.sheets["Recommended Trades"].write("C1", "Market Capitalization", dollar_format)
##writer.sheets["Recommended Trades"].write("D1", "Number of Shares to Buy", integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [ ]:
column_formats = {
    "A" : ["Ticker", string_format],
    "B" : ["Stock Price", dollar_format],
    "C" : ["Market Capitalization", dollar_format],
    "D" : ["Number of Shares to Buy", integer_format], 
}

for column in column_formats.keys():
    writer.sheets["Recommended Trades"].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets["Recommended Trades"].write(f"{column}1", column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()